In [ ]:
#| default_exp grid

# figure-game-solver

> API details.

In [61]:
#|hide
from nbdev.showdoc import *
from fastcore.utils import *
from fastcore.test import *
from functools import reduce
import time

In [ ]:
#|export
class Grid:
    def __init__(self, initial_value=None):
        if initial_value is not None:
            self.grid = initial_value
        else:
            self.grid=[]
            empty_row=[0,0,0,0,0]
            for i in range(5):
                self.grid.append(empty_row)

    def __repr__(self): 
        return f"Grid({str(self.grid)})"

    def __str__(self):
        s = '\r\n'.join([''.join(['{:3}'.format(item) for item in row]) for row in self.grid])
        return s

In [ ]:
g = Grid()
print(g)

  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0


In [ ]:
@patch
def __eq__(self:Grid,other:Grid):
    return self.grid==other.grid

In [ ]:
test_eq(Grid(),Grid())

In [ ]:
y=1
p=2
b=3
w=4
g = Grid([[y,y,p,y,b],[p,p,w,b,w],[y,p,y,w,b],[b,p,w,w,y],[w,b,w,w,b]])
print(g)

  1  1  2  1  3
  2  2  4  3  4
  1  2  1  4  3
  3  2  4  4  1
  4  3  4  4  3


In [ ]:
test_ne(g,Grid())

Add a copy operation so that we can be immutable.

In [ ]:
@patch
def copy(self:Grid):
    return Grid([row[:] for row in self.grid])

Try mutating a copy, and check that the original doesn't change.

In [ ]:
h = g.copy()
test_ne(g.grid[0][0],0)
h.grid[0][0]=0
test_eq(h.grid[0][0],0)
test_ne(g.grid[0][0],0)


## Grid operations

We need to be able to pick a column. 
When that column is picked, we need to remove the item in the bottom row of that column and any connected items.
Then all the remaining items should fall downward in their column.


### Remove

We start with the remove operation.
First of all, a mutational function that is not exported -- just used internally.

In [ ]:
def remove_recursive(grid, i:int, j:int, value:int):
    grid[i][j]=0
    if i>0 and grid[i-1][j]==value:
        remove_recursive(grid,i-1,j,value)
    if i<4 and grid[i+1][j]==value:
        remove_recursive(grid,i+1,j,value)
    if j>0 and grid[i][j-1]==value:
        remove_recursive(grid,i,j-1,value)
    if j<4 and grid[i][j+1]==value:
        remove_recursive(grid,i,j+1,value)

In [ ]:
@patch
def remove(self:Grid, column:int):
    value = self.grid[4][column]
    if value==0:
        return self
    result = self.copy()
    remove_recursive(result.grid,4,column,result.grid[4][column])
    return result

Test the remove method.

In [ ]:
print(g)
print()
print(g.remove(0))
print()
print(g.remove(3))

  1  1  2  1  3
  2  2  4  3  4
  1  2  1  4  3
  3  2  4  4  1
  4  3  4  4  3

  1  1  2  1  3
  2  2  4  3  4
  1  2  1  4  3
  3  2  4  4  1
  0  3  4  4  3

  1  1  2  1  3
  2  2  4  3  4
  1  2  1  0  3
  3  2  0  0  1
  4  3  0  0  3


In [ ]:
h=g.remove(3)
print(h)

  1  1  2  1  3
  2  2  4  3  4
  1  2  1  0  3
  3  2  0  0  1
  4  3  0  0  3


### Fall

For each cell, choose the next highest non-zero item.

In [ ]:
@patch
def fall(self:Grid):
    result = self.copy()
    for j in range(5):
        for i in range(4,-1,-1):
            # Find first non-zero value at or above row i
            if result.grid[i][j]==0:
                for k in range(i-1,-1,-1):
                    if (result.grid[k][j]!=0):
                        result.grid[i][j]=result.grid[k][j]
                        result.grid[k][j]=0
                        break
    return result

In [ ]:
j = h.fall()
print(j)


  1  1  0  0  3
  2  2  0  0  4
  1  2  2  0  3
  3  2  4  1  1
  4  3  1  3  3


In [ ]:
@patch
def pick(self:Grid, column:int):
    removed = self.remove(column)
    fallen = removed.fall()
    return fallen

In [ ]:
h = g.pick(3)
print(h)


  1  1  0  0  3
  2  2  0  0  4
  1  2  2  0  3
  3  2  4  1  1
  4  3  1  3  3


A grid of all zeroes should have `is_zero()` return True.

In [ ]:
@patch
def is_zero(self:Grid):
    for i in range(5):
        for j in range(5):
            if self.grid[i][j] != 0:
                return False
    return True

In [ ]:
test_eq(Grid().is_zero(),True)

In [ ]:
@patch
def picks(self:Grid,columns):
    return reduce(lambda a,b : a.pick(b), columns, self)

In [ ]:
h = g.picks([4,3,1,1])
print(h)

  1  0  0  0  0
  2  0  0  0  0
  1  0  0  0  3
  3  0  4  1  3
  4  1  1  3  1


In [ ]:
g_pick_4311_expected = Grid([[1,0,0,0,0],[2,0,0,0,0],[1,0,0,0,3],[3,0,4,1,3],[4,1,1,3,1]])
assert(h.grid == g_pick_4311_expected.grid)

## Solver

Now the grid class is working, we can start looking at how to construct a solution. The first thing to try is a breadth-first search.

In [ ]:
g0 = Grid()
print(g0)
print(g0.is_zero())

  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
True


In [ ]:
g1 = Grid([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,3,0,0]])
print(g1)

  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  3  0  0


In [ ]:
g2 = Grid([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,3,2,0]])
print(g2)

  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  3  2  0


In [ ]:
def search(grid:Grid, columns:list):
    queue = [columns]
    while len(queue)>0:
        candidate = queue.pop(0)
        #print(f"Candidate: {candidate}")
        result = grid.picks(candidate)
        if result.is_zero():
            return candidate
        for j in range(5):
            if result.grid[4][j] != 0:
                next = candidate + [j]
                queue.append(next)
    

In [ ]:
result = search(g0,[])
print(result)

Candidate: []
[]


In [ ]:
result = search(g1,[])
print(result)

Candidate: []
Candidate: [2]
[2]


In [ ]:
result = search(g2,[2])
print(result)

Candidate: [2]
Candidate: [2, 3]
[2, 3]


In [62]:
print(g)
start = time.time()
result = search(g,[])
end = time.time()

print(f"Found {result} in {end-start} seconds")


  1  1  2  1  3
  2  2  4  3  4
  1  2  1  4  3
  3  2  4  4  1
  4  3  4  4  3
Found [0, 0, 4, 2, 1, 3, 0, 2, 4] in 69.04775381088257 seconds
